In [18]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import xgboost as xgb
from xgboost import plot_importance
# import warnings
# warnings.filterwarnings('ignore')

# Set display options
# pd.set_option('display.max_columns', None)

In [19]:
# Load datasets
train_data = pd.read_csv('enriched_data_train.csv')
test_data = pd.read_csv('enriched_data_test.csv')

# Examine the training data
print("Training data shape:", train_data.shape)
print("\nTraining data info:")
train_data.info()

# Check for missing values
print("\nMissing values in training data:")
print(train_data.isnull().sum().sum())
print("\nMissing values in test data:")
print(test_data.isnull().sum().sum())

In [20]:
# Define the target variable
target = 'TotalExpense'

# Remove HH_ID from features as it's an identifier
X_train = train_data.drop(['HH_ID', target], axis=1)
y_train = train_data[target]

X_test = test_data.drop(['HH_ID', target], axis=1)
y_test = test_data[target]

# Convert any categorical variables to numerical
# For this dataset, we already verified most are numerical, but including for robustness
# X_train = pd.get_dummies(X_train)
# X_test = pd.get_dummies(X_test)

# Ensure X_test has the same columns as X_train
missing_cols = set(X_train.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0
    
# Ensure columns order matches
X_test = X_test[X_train.columns]

In [21]:
# Apply normalization using StandardScaler
scaler = StandardScaler()

# Fit the scaler on training data only to prevent data leakage
X_train_scaled = scaler.fit_transform(X_train)

# Transform test data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame to keep column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Display summary statistics before and after scaling
print("Before scaling - Training data statistics:")
print(X_train.describe().loc[['mean', 'std']].T.head())
print("\nAfter scaling - Training data statistics:")
print(X_train_scaled.describe().loc[['mean', 'std']].T.head())

In [22]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [23]:
# Train a basic XGBoost model with normalized data
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

model.fit(X_train_scaled, y_train_log)

# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred = np.expm1(y_pred)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Absolute Percentage Error: {mape:.2f}")
print(f"R² Score: {r2:.4f}")

In [24]:
# Plot feature importance
plt.figure(figsize=(12, 8))
xgb.plot_importance(model, max_num_features=15, height=0.8)
plt.title('Top 15 Features by Importance')
plt.tight_layout()
plt.show()

# Get feature importance as DataFrame for better analysis
feature_importance = model.feature_importances_
feature_names = X_train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

print("Top 15 Important Features:")
importance_df.head(15)

In [ ]:
# Define an improved parameter grid for tuning based on initial results
improved_param_grid = {
    # Current best is n_estimators=200, so explore around that value
    'n_estimators': [150, 200, 250, 300],
    # Current best is learning_rate=0.2, so refine around that value
    'learning_rate': [0.15, 0.2, 0.25],
    # Current best is max_depth=6, so explore around that
    'max_depth': [5, 6, 7, 8],
    # Add more hyperparameters that weren't in the initial grid
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 1.0],
    'reg_lambda': [0.1, 1.0, 10.0]
}

# Use RandomizedSearchCV instead of GridSearchCV for more efficient exploration
from sklearn.model_selection import RandomizedSearchCV

# Define XGBoost model with the best parameters from initial search as starting point
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    learning_rate=0.2,
    max_depth=6,
    random_state=42
)

# Set up RandomizedSearchCV with cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=improved_param_grid,
    n_iter=50,  # Number of parameter settings sampled
    scoring='neg_mean_squared_error',
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit the random search
random_search.fit(X_train_scaled, y_train_log)

# Best parameters and score
print("Best parameters:", random_search.best_params_)
print("Best RMSE:", np.sqrt(-random_search.best_score_))

# Use the best model for predictions
best_random_model = random_search.best_estimator_
y_pred_random = best_random_model.predict(X_test_scaled)
y_pred_random = np.expm1(y_pred_random)

# Evaluate the improved tuned model
mse_random = mean_squared_error(y_test, y_pred_random)
rmse_random = np.sqrt(mse_random)
r2_random = r2_score(y_test, y_pred_random)
mape_random = mean_absolute_percentage_error(y_test, y_pred_random)

print(f"\nImproved Model Results:")
print(f"Root Mean Squared Error: {rmse_random:.2f}")
print(f"R² Score: {r2_random:.4f}")
print(f"Mean Absolute Percentage Error: {mape_random:.2f}")

# Compare with previous best model
print(f"\nImprovement over previous model:")
print(f"RMSE improvement: {rmse_best - rmse_random:.2f} ({(rmse_best - rmse_random)/rmse_best*100:.2f}%)")
print(f"R² improvement: {r2_random - r2_best:.4f} ({(r2_random - r2_best)*100:.2f}%)")

In [ ]:
# Fine-tuning with a focused grid based on RandomizedSearchCV results
# This cell should be run after examining the random search results

# Replace these values with the best parameters from random search
best_n_estimators = random_search.best_params_.get('n_estimators')
best_learning_rate = random_search.best_params_.get('learning_rate')
best_max_depth = random_search.best_params_.get('max_depth')
best_min_child_weight = random_search.best_params_.get('min_child_weight')
best_gamma = random_search.best_params_.get('gamma')
best_subsample = random_search.best_params_.get('subsample')
best_colsample_bytree = random_search.best_params_.get('colsample_bytree')
best_reg_alpha = random_search.best_params_.get('reg_alpha')
best_reg_lambda = random_search.best_params_.get('reg_lambda')

# Create a focused grid search around the best parameters
focused_param_grid = {
    'n_estimators': [best_n_estimators-50, best_n_estimators, best_n_estimators+50],
    'learning_rate': [best_learning_rate*0.9, best_learning_rate, best_learning_rate*1.1],
    'max_depth': [max(1, best_max_depth-1), best_max_depth, best_max_depth+1],
    'min_child_weight': [max(1, best_min_child_weight-1), best_min_child_weight, best_min_child_weight+1],
    'gamma': [max(0, best_gamma-0.1), best_gamma, min(1, best_gamma+0.1)],
    'subsample': [max(0.5, best_subsample-0.1), best_subsample, min(1, best_subsample+0.1)],
    'colsample_bytree': [max(0.5, best_colsample_bytree-0.1), best_colsample_bytree, min(1, best_colsample_bytree+0.1)]
}

# Set up final GridSearchCV with the focused parameters
final_grid_search = GridSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:squarederror', random_state=42, reg_alpha=best_reg_alpha, reg_lambda=best_reg_lambda),
    param_grid=focused_param_grid,
    scoring='neg_mean_squared_error',
    cv=cv,
    verbose=1,
    n_jobs=-1
)

# Fit the final grid search
final_grid_search.fit(X_train_scaled, y_train_log)

# Best parameters and score
print("Final best parameters:", final_grid_search.best_params_)
print("Final best RMSE:", np.sqrt(-final_grid_search.best_score_))

# Use the final best model for predictions
final_best_model = final_grid_search.best_estimator_
y_pred_final = final_best_model.predict(X_test_scaled)
y_pred_final = np.expm1(y_pred_final)

# Evaluate the final model
mse_final = mean_squared_error(y_test, y_pred_final)
rmse_final = np.sqrt(mse_final)
r2_final = r2_score(y_test, y_pred_final)
mape_final = mean_absolute_percentage_error(y_test, y_pred_final)

print(f"\nFinal Model Results:")
print(f"Root Mean Squared Error: {rmse_final:.2f}")
print(f"R² Score: {r2_final:.4f}")
print(f"Mean Absolute Percentage Error: {mape_final:.2f}")

In [ ]:
# Plot learning curves to check for overfitting/underfitting
from sklearn.model_selection import learning_curve

# Use the best model parameters from final tuning
best_params = final_grid_search.best_params_
model_for_learning_curve = xgb.XGBRegressor(
    objective='reg:squarederror',
    **best_params,
    reg_alpha=best_reg_alpha,
    reg_lambda=best_reg_lambda,
    random_state=42
)

# Calculate learning curves
train_sizes, train_scores, test_scores = learning_curve(
    model_for_learning_curve,
    X_train_scaled,
    y_train_log,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Calculate mean and standard deviation for training scores and test scores
train_mean = -np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = -np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot learning curve
plt.figure(figsize=(10, 6))
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='b')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='r')
plt.plot(train_sizes, train_mean, 'o-', color='b', label='Training score')
plt.plot(train_sizes, test_mean, 'o-', color='r', label='Cross-validation score')
plt.title('Learning Curves for XGBoost')
plt.xlabel('Training Examples')
plt.ylabel('MSE')
plt.legend(loc='best')
plt.grid(True)
plt.show()

# Transform MSE to RMSE for better interpretability
train_rmse = np.sqrt(train_mean)
test_rmse = np.sqrt(test_mean)

# Plot RMSE learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_rmse, 'o-', color='b', label='Training RMSE')
plt.plot(train_sizes, test_rmse, 'o-', color='r', label='Cross-validation RMSE')
plt.title('Learning Curves (RMSE) for XGBoost')
plt.xlabel('Training Examples')
plt.ylabel('RMSE')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
# Plot actual vs predicted values using the final best model
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_final, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted TotalExpense (Final Model)')
plt.tight_layout()
plt.show()

# Distribution of errors
errors = y_test - y_pred_final
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.title('Distribution of Prediction Errors (Final Model)')
plt.tight_layout()
plt.show()

In [ ]:
# Save the final best model and the scaler
final_best_model.save_model('xgboost_household_expenditure_model_final.json')
import pickle
with open('feature_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
print("Final model saved successfully!")
print("Feature scaler saved successfully!")

# Get feature importance for the final model
feature_importance_final = final_best_model.feature_importances_
feature_names = X_train.columns
importance_df_final = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance_final})
importance_df_final = importance_df_final.sort_values('Importance', ascending=False).reset_index(drop=True)

# Save feature importance for future reference
importance_df_final.to_csv('feature_importance_final.csv', index=False)
print("Feature importance saved successfully!")

# Display top features
print("\nTop 15 Important Features (Final Model):")
display(importance_df_final.head(15))

# Plot feature importance for final model
plt.figure(figsize=(12, 8))
xgb.plot_importance(final_best_model, max_num_features=15, height=0.8)
plt.title('Top 15 Features by Importance (Final Model)')
plt.tight_layout()
plt.show()